## <span style="color:#1a73e8">Houskeeping: Load packages and datasets, clean dataframes, etc.</span>

In [2]:
library(lme4)
library(simr)
library(dplyr)
library(lmerTest)
library(ggplot2)
library(sjPlot)
library(interactions)
library(emmeans)
library(car)
library(patchwork)
library(psych)
library(corrplot)
library(forcats)
library(performance)
library(ggeffects)
library(patchwork)

In [74]:
# Adjust according to your file path
file_path <- "../data/wm_task_questionnaires"

# Dataset with only incorrect trials for error type analysis
file_path_error <- "../data/df_errors.csv"

In [77]:
# Select the columns we need for the analysis
df <- read.csv(file_path) %>%
  select(
    matches("^vviq[-.]\\d+$|^irq[-.]visual[-.]\\d+$|^irq[-.]verbal[-.]\\d+$"), 
    it_sim_dis_diff, it_sim_dis_diff_sq, reliability, v2_sim_dis_diff, v2_sim_dis_diff_sq,
    participant, validity, resp_correct, Accuracy, validity_binary, reliability_binary,
    v2_sim_dis_attend, v2_sim_dis_attend_z, v2_sim_dis_attend_sq_z, it_sim_dis_attend,
    it_sim_dis_attend_z, it_sim_dis_attend_sq_z, v2_sim_dis_unattend, v2_sim_dis_unattend_z,
    v2_sim_dis_unattend_sq_z, it_sim_dis_unattend, it_sim_dis_unattend_z,
    it_sim_dis_unattend_sq_z, it_sim_dis_diff_z, v2_sim_dis_diff_z, it_sim_dis_diff_sq_z,
    v2_sim_dis_diff_sq_z, v2_sim_dis_test_z, it_sim_dis_test_z, it_sim_dis_diff_test,
    it_sim_dis_diff_test_sq, v2_sim_dis_diff_test, v2_sim_dis_diff_test_z, it_sim_dis_diff_test_z,
    v2_sim_dis_diff_test_sq_z, it_sim_dis_diff_test_sq_z, it_sim_dis_test_sq_z, validity_binary_z,
    reliability_binary_z, v2_sim_dis_test_sq_z, validity_reliability_z, rt, tested_memorability_resmem_z, it_pos_neg_z,
    v2_pos_neg_z, it_sim_dis_diff_test_sign_z,
    it_sim_dis_diff_test_sign_sq_z, v2_sim_dis_diff_test_sign_z, v2_sim_dis_diff_test_sign_sq_z,
    it_sim_dis_test_sign_z, it_sim_dis_test_sign_sq_z, v2_sim_dis_test_sign_z,
    v2_sim_dis_test_sign_sq_z, it_int_rel, v2_int_rel, it_int_rel_sq,
    v2_int_rel_sq, it_pos_neg, v2_pos_neg, vviq_sum, vviq_z, osivq_verbal_mean,
    z_osivq_verbal_mean, osivq_visual_mean, z_osivq_visual_mean, osivq_spatial_mean,
    z_osivq_spatial_mean, irq_verbal_mean, z_irq_verbal_mean, irq_visual_mean,
    z_irq_visual_mean, pilot_number, osivq.catch.2, irq.catch.1, IT_im1_im2, V2_im1_im2, v2_converges,failed_check_count
  )

In [78]:
# Check participant number
length(unique(df$participant))

[1] 246

In [79]:
df_errors <- read.csv(file_path_error)

In [81]:
length(unique(df_errors$participant))

[1] 261

In [82]:
# Remove participants who failed attention checks from error dataset
df <- df[df$failed_check_count != 3, ]
df_errors <- df_errors[!(df_errors$failed_check_count %in% c(2, 3)), ]

In [83]:
# Check participant number
length(unique(df$participant))

[1] 246

In [84]:
# Square the columns and create new ones
df$IT_im1_im2_sq <- df$IT_im1_im2^2
df$V2_im1_im2_sq <- df$V2_im1_im2^2
library(dplyr)

df <- df %>%
  mutate(
    IT_im1_im2_sq = IT_im1_im2^2,
    V2_im1_im2_sq = V2_im1_im2^2
  )

In [85]:
# Square the columns and create new ones
df_errors$IT_im1_im2_sq <- df_errors$IT_im1_im2^2
df_errors$V2_im1_im2_sq <- df_errors$V2_im1_im2^2

df_errors <- df_errors %>%
  mutate(
    IT_im1_im2_sq = IT_im1_im2^2,
    V2_im1_im2_sq = V2_im1_im2^2
  )

In [86]:
# Create categorical questionnaire variables

df <- df %>%
  mutate(
    vviq_cat = case_when(
      vviq_sum <= quantile(vviq_sum, 1/3, na.rm = TRUE) ~ "low",
      vviq_sum <= quantile(vviq_sum, 2/3, na.rm = TRUE) ~ "medium",
      TRUE ~ "high"
    ),
    osivq_visual_cat = case_when(
      osivq_visual_mean <= quantile(osivq_visual_mean, 1/3, na.rm = TRUE) ~ "low",
      osivq_visual_mean <= quantile(osivq_visual_mean, 2/3, na.rm = TRUE) ~ "medium",
      TRUE ~ "high"
    ),
    osivq_verbal_cat = case_when(
      osivq_verbal_mean <= quantile(osivq_verbal_mean, 1/3, na.rm = TRUE) ~ "low",
      osivq_verbal_mean <= quantile(osivq_verbal_mean, 2/3, na.rm = TRUE) ~ "medium",
      TRUE ~ "high"
    ),
    osivq_spatial_cat = case_when(
      osivq_spatial_mean <= quantile(osivq_spatial_mean, 1/3, na.rm = TRUE) ~ "low",
      osivq_spatial_mean <= quantile(osivq_spatial_mean, 2/3, na.rm = TRUE) ~ "medium",
      TRUE ~ "high"
    ),
    irq_visual_cat = case_when(
      irq_visual_mean <= quantile(irq_visual_mean, 1/3, na.rm = TRUE) ~ "low",
      irq_visual_mean <= quantile(irq_visual_mean, 2/3, na.rm = TRUE) ~ "medium",
      TRUE ~ "high"
    ),
    irq_verbal_cat = case_when(
      irq_verbal_mean <= quantile(irq_verbal_mean, 1/3, na.rm = TRUE) ~ "low",
      irq_verbal_mean <= quantile(irq_verbal_mean, 2/3, na.rm = TRUE) ~ "medium",
      TRUE ~ "high"
    )
  )

In [87]:
# Set reference levels to low for each categorically coded imagery variable for regression models

df <- df %>%
  mutate(
    vviq_cat = fct_relevel(vviq_cat, "low"),
    osivq_visual_cat = fct_relevel(osivq_visual_cat, "low"),
    osivq_verbal_cat = fct_relevel(osivq_verbal_cat, "low"),
    osivq_spatial_cat = fct_relevel(osivq_spatial_cat, "low"),
    irq_visual_cat = fct_relevel(irq_visual_cat, "low"),
    irq_verbal_cat = fct_relevel(irq_verbal_cat, "low")
  )

In [88]:
# Reorder factor levels in dataframe

df <- df %>%
  mutate(
    vviq_cat = fct_relevel(vviq_cat, "low", "medium", "high"),
    irq_visual_cat = fct_relevel(irq_visual_cat, "low", "medium", "high"),
    irq_verbal_cat = fct_relevel(irq_verbal_cat, "low", "medium", "high")
  )

In [89]:
# z_score IT & V2 im1_im2

df$IT_im1_im2_z <- scale(df$IT_im1_im2)
df$V2_im1_im2_z <- scale(df$V2_im1_im2)
df$IT_im1_im2_sq_z <- scale(df$IT_im1_im2_sq)
df$V2_im1_im2_sq_z <- scale(df$V2_im1_im2_sq)

In [90]:
df_errors$IT_im1_im2_z <- scale(df_errors$IT_im1_im2)
df_errors$V2_im1_im2_z <- scale(df_errors$V2_im1_im2)
df_errors$IT_im1_im2_sq_z <- scale(df_errors$IT_im1_im2_sq)
df_errors$V2_im1_im2_sq_z <- scale(df_errors$V2_im1_im2_sq)


In [91]:
# Create df with only correct trials for RT analysis
df_correct <- subset(df, resp_correct == 1)

In [92]:
df_correct <- subset(df, resp_correct == 1)
    df_correct$it_sim_dis_test_z <- scale(df_correct$it_sim_dis_test_z)
    df_correct$it_sim_dis_test_sq_z <- scale(df_correct$it_sim_dis_test_sq_z)
    df_correct$it_sim_dis_diff_test_z <- scale(df_correct$it_sim_dis_diff_test_z)
    df_correct$it_sim_dis_diff_test_sq_z <- scale(df_correct$it_sim_dis_diff_test_sq_z)
    
    df_correct$validity_reliability_z <- scale(df_correct$validity_reliability_z)
    
    df_correct$v2_sim_dis_test_z <- scale(df_correct$v2_sim_dis_test_z)
    df_correct$v2_sim_dis_test_sq_z <- scale(df_correct$v2_sim_dis_test_sq_z)
    df_correct$v2_sim_dis_diff_test_z <- scale(df_correct$v2_sim_dis_diff_test_z)
    df_correct$v2_sim_dis_diff_test_sq_z <- scale(df_correct$v2_sim_dis_diff_test_sq_z)

    df_correct$it_sim_dis_diff_test_sign_z <- scale(df_correct$it_sim_dis_diff_test_sign_z)
    df_correct$it_sim_dis_diff_test_sign_sq_z <- scale(df_correct$it_sim_dis_diff_test_sign_sq_z)
    df_correct$v2_sim_dis_diff_test_sign_z <- scale(df_correct$v2_sim_dis_diff_test_sign_z)
    df_correct$v2_sim_dis_diff_test_sign_sq_z <- scale(df_correct$v2_sim_dis_diff_test_sign_sq_z)

    df_correct$tested_memorability_resmem_z <- scale(df_correct$tested_memorability_resmem_z)

    df_correct$IT_im1_im2_z <- scale(df_correct$IT_im1_im2)
    df_correct$V2_im1_im2_z <- scale(df_correct$V2_im1_im2)
    df_correct$IT_im1_im2_sq_z <- scale(df_correct$IT_im1_im2_sq_z)
    df_correct$V2_im1_im2_sq_z <- scale(df_correct$V2_im1_im2_sq_z)

In [93]:
df_errors$it_sim_dis_test_z <- scale(df_errors$it_sim_dis_test_z)
df_errors$it_sim_dis_test_sq_z <- scale(df_errors$it_sim_dis_test_sq_z)
df_errors$it_sim_dis_diff_test_z <- scale(df_errors$it_sim_dis_diff_test_z)
df_errors$it_sim_dis_diff_test_sq_z <- scale(df_errors$it_sim_dis_diff_test_sq_z)

df_errors$validity_reliability_z <- scale(df_errors$validity_reliability_z)
    
df_errors$v2_sim_dis_test_z <- scale(df_errors$v2_sim_dis_test_z)
df_errors$v2_sim_dis_test_sq_z <- scale(df_errors$v2_sim_dis_test_sq_z)
df_errors$v2_sim_dis_diff_test_z <- scale(df_errors$v2_sim_dis_diff_test_z)
df_errors$v2_sim_dis_diff_test_sq_z <- scale(df_errors$v2_sim_dis_diff_test_sq_z)

df_errors$it_sim_dis_diff_test_sign_z <- scale(df_errors$it_sim_dis_diff_test_sign_z)
df_errors$it_sim_dis_diff_test_sign_sq_z <- scale(df_errors$it_sim_dis_diff_test_sign_sq_z)
df_errors$v2_sim_dis_diff_test_sign_z <- scale(df_errors$v2_sim_dis_diff_test_sign_z)
df_errors$v2_sim_dis_diff_test_sign_sq_z <- scale(df_errors$v2_sim_dis_diff_test_sign_sq_z)

df_errors$tested_memorability_resmem_z <- scale(df_errors$tested_memorability_resmem_z)

df_errors$IT_im1_im2_z <- scale(df_errors$IT_im1_im2)
df_errors$V2_im1_im2_z <- scale(df_errors$V2_im1_im2)
df_errors$IT_im1_im2_sq_z <- scale(df_errors$IT_im1_im2_sq_z)
df_errors$V2_im1_im2_sq_z <- scale(df_errors$V2_im1_im2_sq_z)

In [94]:
# Log transform RTs
df_correct$log_rt <- log(df_correct$rt)
df_errors$log_rt <- log(df_errors$rt)

In [95]:
length(unique(df$participant))

[1] 246

## <span style="color:#1a73e8">Useful functions</span>

<!-- <span style="color:black; font-size:18px">blabla<em></em></span> -->

In [96]:
# A function for forest_plot to visualize effect sizes from regression models, with 95% ci

create_forest_plot <- function(model, title = "Coefficient Forest Plot", 
                               sig_level = 0.055, pos_color = "green4", 
                               neg_color = "red3", nonsig_color = "gray20",
                               plot_width = 4) { 
  library(ggplot2)
  library(dplyr)
  
  # Get model summary
  model_summary <- summary(model)
  
  # Check if the model is lmer or glmer and extract coefficients accordingly
  if ("coefficients" %in% names(model_summary)) {
    coefs <- model_summary$coefficients
  } else if ("coef" %in% names(model_summary)) {
    # Some models might use 'coef' instead of 'coefficients'
    coefs <- model_summary$coef
  } else {
    stop("Could not extract coefficients from the model summary")
  }
  
  coef_names <- rownames(coefs)
  
  # Get the correct column index for p-values
  # lmer models typically have p-values in different columns than glmer
  if (ncol(coefs) >= 5) { # lmer often includes "df" and p-values are column 5
    p_val_col <- 5
  } else { # glmer typically has p-values in column 4
    p_val_col <- 4
  }
  
  # Create a data frame with all the necessary information
  coef_data <- data.frame(
    term = coef_names,
    estimate = coefs[, 1],      # First column contains estimates
    std.error = coefs[, 2],     # Second column contains std errors
    p.value = coefs[, p_val_col] # p-values column
  )
  
  # If p-values are not available (sometimes happens with lmer), calculate them
  if (!("p.value" %in% colnames(coef_data)) || all(is.na(coef_data$p.value))) {
    coef_data$p.value <- 2 * (1 - pnorm(abs(coef_data$estimate / coef_data$std.error)))
  }
  
  # Calculate confidence intervals manually
  coef_data <- coef_data %>%
    mutate(
      conf.low = estimate - 1.96 * std.error,
      conf.high = estimate + 1.96 * std.error,
      # Color based on significance and direction
      color = case_when(
        p.value < sig_level & estimate > 0 ~ pos_color,
        p.value < sig_level & estimate < 0 ~ neg_color,
        TRUE ~ nonsig_color
      )
    ) %>%
    # Filter out the intercept (typically named "(Intercept)" or "Intercept")
    filter(!grepl("^\\(?[Ii]ntercept\\)?$", term))
  
  # Create the forest plot
    p <- ggplot(coef_data, aes(x = estimate, y = reorder(term, estimate))) +
      geom_vline(xintercept = 0, linetype = "dashed", color = "gray50") +
      geom_errorbarh(aes(xmin = conf.low, xmax = conf.high, color = color), 
                     height = 0, linewidth = 0.5) +
      geom_point(aes(color = color), size = 4) +
      scale_color_identity() +
      labs(
        title = title,
        x = "Estimate",
        y = "Fixed Effects"
      ) +
      theme_clean_nogrid +
      theme(
        axis.text.y = element_text(size = 16, angle = 15, hjust = 1),  # ← tilt y labels
        axis.text.x = element_text(size = 14),
        axis.title = element_text(size = 18),
        plot.title = element_text(size = 20, face = "bold", hjust = 0.5, margin = margin(b = 15)),
        plot.margin = margin(t = 20, r = 20, b = 20, l = 20)
      ) +
      coord_cartesian(clip = "off", expand = TRUE)  # prevent clipping
  
  # Return the plot
  return(p)
}

In [97]:
# A function for model_plot(type=pred) for various models I'll be running: (plot_model_predictions)

update_geom_defaults("line", list(linewidth = 3))

# Custom clean theme
theme_clean_nogrid <- theme(
  panel.background = element_blank(),
  plot.background = element_blank(),
  panel.grid = element_blank(),
  axis.line = element_line(color = "black"),
  axis.text = element_text(size = 20),
  axis.title = element_text(size = 20, face = "bold"),
  plot.title = element_text(size = 20, face = "bold", hjust = 0.5),
  strip.text = element_text(size = 20, face = "bold"),
  legend.title = element_text(size = 16),
  legend.text = element_text(size = 16)
)

# Set1-like colors
set1_colors <- c("#E41A1C", "#377EB8", "#4DAF4A")

# function
plot_model_predictions <- function(model, term_list, titles,
                                   y_label = "Predicted Value", 
                                   ylims = NULL,
                                   legend_labels = c("Low", "Medium", "High"),
                                   legend_title = "Imagery Strength",
                                   caption = "",
                                   shared_x_title = TRUE,
                                   custom_colors = NULL,
                                   x_titles = NULL,
                                   width = 14, height = 8,
                                   preview_width = NULL, preview_height = NULL,
                                   save_path = NULL,
                                   transform = NULL,
                                   ci.lvl = ci.lvl
                                  ) {
  # If custom_colors is NULL, use the default set1_colors
  colors_to_use <- if(is.null(custom_colors)) {
    set1_colors  
  } else {
    custom_colors
  }
  
  # If only one color is provided, replicate it
  if(length(colors_to_use) == 1) {
    colors_to_use <- rep(colors_to_use, 3)  # Create 3 copies of the same color
  }

  if (!is.null(preview_width) && !is.null(preview_height)) {
    options(repr.plot.width = preview_width,
            repr.plot.height = preview_height)
  }
    
  # Define custom shared x-axis title
  shared_x_axis_title <- "My Custom X-Axis Title"

  # Determine x-axis titles
  x_axis_titles <- if (shared_x_title) {
    rep("", length(term_list))
  } else if (!is.null(x_titles)) {
    x_titles
  } else {
    sapply(term_list, function(term) term[1])
  }

  # Generate plots
  plots <- Map(function(term, title, x_title, show_legend) {
    plot_model(
      model,
      type = "pred",
      terms = term,
      ci.lvl = ci.lvl,
      transform = transform,  # for glmer: "plogis"
      title = title,
      axis.title = c(x_title, ifelse(title == titles[[1]], y_label, ""))
    ) +
      scale_color_manual(values = colors_to_use, labels = legend_labels, name = legend_title) +
      scale_fill_manual(values = colors_to_use, labels = legend_labels, name = legend_title) +
      guides(color = guide_legend(direction = "horizontal")) +
      theme_clean_nogrid +
      (if (!is.null(ylims)) ylim(ylims[1], ylims[2]) else NULL) +
      (if (!show_legend) theme(legend.position = "none") else NULL)
  },
  term = term_list,
  title = titles,
  x_title = x_axis_titles,
  show_legend = c(TRUE, rep(FALSE, length(term_list) - 1)))

  # Combine plots
  combined <- wrap_plots(plots) +
    plot_layout(guides = "collect") +
    plot_annotation(
      theme = theme(legend.position = "bottom"),
      caption = caption
    ) &
    theme(plot.caption = element_text(size = 22, face = "bold", hjust = 0.5, margin = margin(t = 22)))

  # Save if needed
  if (!is.null(save_path)) {
    ggsave(save_path, combined, width = width, height = height, dpi = 300)
  }

  return(combined)
}

# # USE CASE
# # Shared labels
# plot_model_predictions(
#   model = my_model,
#   term_list = list(c("x", "z1"), c("x", "z2")),
#   titles = c("Z1 Effect", "Z2 Effect"),
#   y_label = "Predicted RT",
#   caption = "Shared x-axis: x",
#   shared_x_title = TRUE
# )

# # Custom x-axis labels per plot
# plot_model_predictions(
#   model = my_model,
#   term_list = list(c("x1", "z1"), c("x2", "z2")),
#   titles = c("X1 × Z1", "X2 × Z2"),
#   x_titles = c("X1 Similarity", "X2 Similarity"),
#   y_label = "Predicted RT",
#   caption = "Different predictors with custom X labels",
#   shared_x_title = FALSE
# )
       
# # GLMER USE CASE
# plot_model_predictions(
#   model = my_glmer_model,
#   term_list = list(c("x", "z1"), c("x", "z2")),
#   titles = c("Z1", "Z2"),
#   y_label = "Predicted Accuracy",
#   caption = "Predicted Probabilities from GLMM",
#   transform = "plogis",  # <- This is important
#   preview_width = 12, preview_height = 8
# )

In [98]:
# A function for model_plot(type=eff) for various models I'll be running: (plot_model_effects)

update_geom_defaults("line", list(linewidth = 3))

# Custom clean theme
theme_clean_nogrid <- theme(
  panel.background = element_blank(),
  plot.background = element_blank(),
  panel.grid = element_blank(),
  axis.line = element_line(color = "black"),
  axis.text = element_text(size = 24),
  axis.title = element_text(size = 24, face = "bold"),
  plot.title = element_text(size = 24, face = "bold", hjust = 0.5),
  strip.text = element_text(size = 24, face = "bold"),
  legend.title = element_text(size = 24),
  legend.text = element_text(size = 22)
)

# Function to plot marginal effects
plot_model_effects <- function(model, term_list, titles,
                                y_label = "Estimated Marginal Effects",
                                ylims = NULL,
                                legend_labels = c("Low", "Medium", "High"),
                                legend_title = "Imagery Strength",
                                caption = "",
                                shared_x_title = TRUE,
                                custom_colors = NULL,
                                x_titles = NULL,
                                width = 14, height = 8,
                                preview_width = NULL, preview_height = NULL,
                                save_path = NULL,
                                ci.lvl = 0.68,  
                                legend_position = "right",  # Customizable legend position
                                legend_direction = "vertical",  # Vertical or horizontal
                                transform = NULL
                                ) {
  # Set default colors if no custom colors are provided
  colors_to_use <- if (is.null(custom_colors)) {
    set1_colors  
  } else {
    custom_colors
  }
  
  # Handle case where only one color is provided
  if (length(colors_to_use) == 1) {
    colors_to_use <- rep(colors_to_use, 3)
  }
  
  # Set preview size if requested
  if (!is.null(preview_width) && !is.null(preview_height)) {
    options(repr.plot.width = preview_width,
            repr.plot.height = preview_height)
  }

  # Define x-axis titles dynamically
  x_axis_titles <- if (shared_x_title) {
    rep(ifelse(is.null(x_titles), "X-Axis", x_titles[[1]]), length(term_list))
  } else if (!is.null(x_titles)) {
    x_titles
  } else {
    sapply(term_list, function(term) term[1])
  }

  # Conditionally adjust show_legend based on the number of terms
  show_legend <- if (length(term_list) == 1) TRUE else c(FALSE, TRUE)

  # Generate the plots
  plots <- Map(function(term, title, x_title, show_legend_flag) {
    plot_model(
      model,
      type = "eff",  # Generate marginal effects plots
      terms = term,
      ci.lvl = ci.lvl,
      title = title,
      axis.title = c(x_title, ifelse(title == titles[[1]], y_label, "")),
      show.legend = show_legend_flag,  # Control legend visibility
      transform = transform
    ) +
      scale_color_manual(
          values = if (!is.null(custom_colors)) custom_colors else colors_to_use,
          labels = legend_labels,
          name = legend_title
      ) +
      scale_fill_manual(values = colors_to_use, labels = legend_labels, name = legend_title) +
      guides(color = guide_legend(direction = legend_direction)) +
      theme_clean_nogrid +
      (if (!is.null(ylims)) ylim(ylims[1], ylims[2]) else NULL) +
      (if (!show_legend_flag) theme(legend.position = "none") else NULL)
  },
  term = term_list,
  title = titles,
  x_title = x_axis_titles,
  show_legend_flag = show_legend)

  # If only one plot, no need for layout
  if (length(term_list) == 1) {
    combined <- plots[[1]]
  } else {
    # Combine plots and collect legend only from the last plot
    combined <- wrap_plots(plots) +
      plot_layout(guides = "collect") & 
      theme(
        legend.position = legend_position,
        plot.caption = element_text(size = 22, face = "bold", hjust = 0.5, margin = margin(t = 10))
      )
  }

  # Add a caption below the plot
  combined <- combined +
    plot_annotation(
      caption = caption,
      theme = theme(plot.caption = element_text(size = 22, face = "bold", hjust = 0.5, margin = margin(t = 10)))
    )

  # Save the plot 
  if (!is.null(save_path)) {
    ggsave(save_path, combined, width = width, height = height, dpi = 300)
  }

  # Return the combined plot
  return(combined)
}


In [107]:
# A function to create model summary tables

model_summary_table <- function(model,
                                caption = "Model Summary",
                                p_cutoff = 0.055,
                                rename_terms = NULL,
                                font_size = 16,
                                caption_size = 20) {
  require(parameters)
  require(dplyr)
  require(kableExtra)
  require(knitr)
  require(IRdisplay)
  require(plyr)

  # Check if binomial model
  is_logistic <- inherits(model, "glmerMod") &&
                 grepl("binomial", family(model)$family, ignore.case = TRUE)

  # Get parameters (exponentiated for logit)
  param_table <- model_parameters(
    model,
    effects = "fixed",
    ci = NULL,
    digits = 3,
    exponentiate = is_logistic
  )

  # Rename for logistic models
  if (is_logistic) {
    names(param_table)[names(param_table) == "Coefficient"] <- "Odds Ratio"
  }

  # Optional renaming of predictors
  if (!is.null(rename_terms) && is.character(rename_terms) && !is.null(names(rename_terms))) {
    param_table$Parameter <- plyr::revalue(param_table$Parameter, rename_terms)
  }

  # Add significance stars
  param_table <- param_table %>%
    mutate(Signif = case_when(
      p < 0.001 ~ "***",
      p < 0.01  ~ "**",
      p < p_cutoff ~ "*",
      p < 0.1   ~ ".",
      TRUE      ~ ""
    ))

  # Bold significant parameter names
  param_table <- param_table %>%
    mutate(Parameter = ifelse(p < p_cutoff,
                              cell_spec(Parameter, bold = TRUE),
                              as.character(Parameter)))

  # Determine columns to show (check existence)
  coef_col <- if ("Odds Ratio" %in% names(param_table)) "Odds Ratio" else "Coefficient"
  stat_col <- if ("t" %in% names(param_table)) "t" else if ("z" %in% names(param_table)) "z" else NULL

  # Final columns to display
  display_cols <- c("Parameter", coef_col, "SE", stat_col, "p", "Signif")
  display_cols <- display_cols[display_cols %in% names(param_table)]

  # Format and display table
  html_table <- param_table %>%
    select(all_of(display_cols)) %>%
    kbl(escape = FALSE, format = "html", caption = caption) %>%
    kable_styling(
      full_width = FALSE,
      position = "center",
      bootstrap_options = c("striped", "hover", "condensed", "responsive"),
      font_size = font_size
    ) %>%
    as.character()

  styled_html <- paste0(
    "<style>caption { font-size:", caption_size, "px; font-weight: bold; text-align: center; }</style>",
    html_table
  )

  IRdisplay::display_html(styled_html)
}


# # Use case
# term_names <- c(
#   "v2_sim_dis_test_z" = "V2 Similarity",
#   "vviq_z" = "VVIQ",
#   "z_irq_visual_mean" = "IRQ Visual",
#   "z_irq_verbal_mean" = "IRQ Verbal",
#   "it_sim_dis_test_z" = "Item Similarity"
# )

# pretty_model_table(
#   model = distractor_tested_similarity_rt_cont_model,
#   caption = "Model Summary: V2 & Item Similarity Effects",
#   p_cutoff = 0.055,
#   rename_terms = term_names,
#   font_size = 16,
#   caption_size = 20
# )

In [108]:
run_imagery_models <- function(visual_var, verbal_var = NULL, model_name_suffix) {
  # Run three models (RT, Accuracy, Errors) for specified imagery variables
  # 
  # Parameters:
  # visual_var: Name of visual imagery variable (e.g., 'z_irq_visual_mean')
  # verbal_var: Name of verbal imagery variable (e.g., 'z_irq_verbal_mean') or NULL
  # model_name_suffix: Suffix for model names (e.g., 'irq', 'osivq', 'vviq')
  # 
  # Returns: List of models and prints summary tables
  
  # Construct the imagery terms based on whether we have verbal measure
  if (is.null(verbal_var)) {
    # For VVIQ (visual only)
    imagery_terms <- paste0(visual_var, " * v2_sim_dis_test_z + ",
                           visual_var, " * validity_binary_z + ",
                           visual_var, " * tested_memorability_resmem_z")
  } else {
    # For IRQ/OSIVQ (both visual and verbal)
    imagery_terms <- paste0("(", visual_var, " + ", verbal_var, ") * v2_sim_dis_test_z + ",
                           "(", visual_var, " + ", verbal_var, ") * validity_binary_z + ",
                           "(", visual_var, " + ", verbal_var, ") * tested_memorability_resmem_z")
  }
  
  # Common terms for all models
  common_terms <- "tested_memorability_resmem_z * v2_sim_dis_test_z + 
                   tested_memorability_resmem_z * validity_binary_z + 
                   v2_sim_dis_test_z * validity_binary_z + 
                   (1 | participant)"
  
  # Full formula
  full_formula <- paste(imagery_terms, "+", common_terms)
  
  # Model 1: RT (log-transformed)
  cat("\n", rep("=", 50), "\n")
  cat("RT MODEL -", toupper(model_name_suffix), "\n")
  cat(rep("=", 50), "\n")
  
  rt_formula <- paste("log_rt ~", full_formula)
  model_rt <- lmer(as.formula(rt_formula), 
                   data = df_correct, 
                   control = lmerControl(optimizer = "bobyqa"))
  
  # Assign to global environment with dynamic name
  assign(paste0(model_name_suffix, "_model_rt"), model_rt, envir = .GlobalEnv)
  
  model_summary_table(
    model = model_rt,
    caption = paste("RT Model -", toupper(model_name_suffix)),
    p_cutoff = 0.05,
    font_size = 16,
    caption_size = 20
  )
  
  # Model 2: Accuracy (binomial)
  cat("\n" , rep("=", 50), "\n")
  cat("ACCURACY MODEL -", toupper(model_name_suffix), "\n")
  cat(rep("=", 50), "\n")
  
  acc_formula <- paste("resp_correct ~", full_formula)
  model_acc <- glmer(as.formula(acc_formula), 
                     data = df, 
                     family = binomial, 
                     control = glmerControl(optimizer = "bobyqa"))
  
  # Assign to global environment with dynamic name
  assign(paste0(model_name_suffix, "_model_acc"), model_acc, envir = .GlobalEnv)
  
  model_summary_table(
    model = model_acc,
    caption = paste("Accuracy Model -", toupper(model_name_suffix)),
    p_cutoff = 0.05,
    font_size = 16,
    caption_size = 20
  )
  
  # Model 3: Same Category Errors (binomial)
  cat("\n", rep("=", 50), "\n")
  cat("ERROR MODEL -", toupper(model_name_suffix), "\n")
  cat(rep("=", 50), "\n")
  
  err_formula <- paste("same_category_error ~", full_formula)
  model_err <- glmer(as.formula(err_formula), 
                     data = df_errors, 
                     family = binomial, 
                     control = glmerControl(optimizer = "bobyqa"))
  
  # Assign to global environment with dynamic name
  assign(paste0(model_name_suffix, "_model_errors"), model_err, envir = .GlobalEnv)
  
  model_summary_table(
    model = model_err,
    caption = paste("Error Model -", toupper(model_name_suffix)),
    p_cutoff = 0.05,
    font_size = 16,
    caption_size = 20
  )
  
  # Return list of models
  models <- list(
    rt = model_rt,
    accuracy = model_acc,
    errors = model_err
  )
  
  return(models)
}

# # Usage Examples:

# # 1. IRQ Models (both visual and verbal)
# cat("RUNNING IRQ MODELS\n")
# cat(rep("#", 60), "\n")
# irq_models <- run_imagery_models(
#   visual_var = "z_irq_visual_mean",
#   verbal_var = "z_irq_verbal_mean", 
#   model_name_suffix = "irq"
# )

# # 2. OSIVQ Models (both visual and verbal)  
# cat("\n\nRUNNING OSIVQ MODELS\n")
# cat(rep("#", 60), "\n")
# osivq_models <- run_imagery_models(
#   visual_var = "z_osivq_visual_mean",
#   verbal_var = "z_osivq_verbal_mean",
#   model_name_suffix = "osivq"  
# )

# # 3. VVIQ Models (visual only)
# cat("\n\nRUNNING VVIQ MODELS\n")
# cat(rep("#", 60), "\n")
# vviq_models <- run_imagery_models(
#   visual_var = "vviq_z",
#   verbal_var = NULL,  # No verbal component for VVIQ
#   model_name_suffix = "vviq"
# )

# # The models are now available as:
# # irq_model_rt, irq_model_acc, irq_model_errors
# # osivq_model_rt, osivq_model_acc, osivq_model_errors  
# # vviq_model_rt, vviq_model_acc, vviq_model_errors

# cat("\n\nAll models completed and saved to global environment!\n")
# cat("Model objects created:\n")
# cat("- IRQ: irq_model_rt, irq_model_acc, irq_model_errors\n")
# cat("- OSIVQ: osivq_model_rt, osivq_model_acc, osivq_model_errors\n") 
# cat("- VVIQ: vviq_model_rt, vviq_model_acc, vviq_model_errors\n")

In [154]:
##############################################################################
## 0 · LOAD / INSTALL PACKAGES  ---------------------------------------------
##############################################################################
pkgs <- c("lme4", "parameters", "dplyr", "kableExtra",
          "stringr", "magick", "pdftools", "IRdisplay")
to_install <- pkgs[!pkgs %in% installed.packages()[, "Package"]]
if (length(to_install)) install.packages(to_install, type = "binary")
invisible(lapply(pkgs, library, character.only = TRUE))

##############################################################################
## 1 · MODEL TABLE — APA style, caption bold + big, PNG = PDF raster
##############################################################################
##############################################################################
## REPLACEMENT for model_summary_table()
##  • caption bold + larger
##  • APA italics
##  • Signif column retained
##  • PDF→PNG so italics survive, no extra_css
##############################################################################
model_summary_table <- function(model,
                                caption        = "Model Summary",
                                p_cutoff       = 0.055,
                                rename_terms   = NULL,
                                font_size      = 16,
                                caption_size   = 20,
                                save_as        = NULL,      # "name.pdf" | "name.png"
                                dpi            = 300) {

  # ---- recognise logistic -----------------------------------------------
  is_logit <- inherits(model, "glmerMod") &&
              grepl("binomial", family(model)$family, ignore.case = TRUE)

  # ---- extract parameters ------------------------------------------------
  tab <- parameters::model_parameters(
    model, effects = "fixed", ci = NULL, digits = 3,
    exponentiate = is_logit
  )
  coef_col <- if (is_logit) "Odds Ratio" else "Coefficient"
  if (is_logit)
    names(tab)[names(tab) == "Coefficient"] <- coef_col

  # ---- rename predictors -------------------------------------------------
  if (!is.null(rename_terms)) {
    pat <- setNames(rename_terms, paste0("\\b", names(rename_terms), "\\b"))
    tab$Parameter <- stringr::str_replace_all(tab$Parameter, pat)
    tab$Parameter <- gsub(":", " × ", tab$Parameter)
  }

  # ---- stars and bolding -------------------------------------------------
  tab <- tab %>%
    mutate(Signif = case_when(
             p < .001 ~ "***",
             p < .01  ~ "**",
             p < p_cutoff ~ "*",
             p < .1  ~ ".",
             TRUE     ~ ""),
           Parameter = ifelse(p < p_cutoff,
                              kableExtra::cell_spec(Parameter, bold = TRUE),
                              Parameter))

  stat_col <- intersect(c("t", "z"), names(tab))[1]
  keep     <- c("Parameter", coef_col, "SE", stat_col, "p", "Signif")

  # ---- APA headers -------------------------------------------------------
  ital  <- function(x) sprintf(
    "<span style='font-style:italic;font-family:Times,serif;'>%s</span>", x)

  hdrs  <- c("Predictor",
             if (is_logit) "OR" else ital("b"),
             ital("SE"),
             ital(stat_col),
             ital("p"),
             "")

  caption_html <- sprintf(
    "<span style='font-size:%dpx;font-weight:bold;'>%s</span>",
    caption_size, caption)

  kbl_obj <- tab %>%
    select(all_of(keep)) %>%
    kableExtra::kbl(escape = FALSE, caption = caption_html,
                    col.names = hdrs, align = "lrrrrc") %>%
    kableExtra::kable_styling(
      full_width = FALSE, position = "center",
      bootstrap_options = c("condensed", "responsive"),
      font_size = font_size
    )

  # ---- export: PDF direct, PNG via PDF raster ----------------------------
  if (!is.null(save_as)) {
    ext <- tools::file_ext(save_as); stopifnot(ext %in% c("pdf", "png"))

    if (ext == "pdf") {
      kableExtra::save_kable(kbl_obj, save_as, density = dpi)
    } else {                          # export PNG
      tmp_pdf <- tempfile(fileext = ".pdf")
      kableExtra::save_kable(kbl_obj, tmp_pdf, density = dpi)
      img <- magick::image_read_pdf(tmp_pdf, density = dpi)
      magick::image_write(img, save_as, format = "png")
      unlink(tmp_pdf)
    }
  }

  # ---- display inside R session -----------------------------------------
  IRdisplay::display_html(as.character(kbl_obj))
}


##############################################################################
## 2 · THREE-WAY INTERACTION WRAPPER
##############################################################################
run_imagery_models_threeway <- function(visual_var,
                                        verbal_var        = NULL,
                                        model_name_suffix,
                                        rename_terms      = NULL,
                                        save_dir          = "tables",
                                        output            = "pdf",  # "pdf" | "png"
                                        dpi               = 300,
                                        p_cutoff          = 0.05,
                                        font_size         = 16,
                                        caption_size      = 20) {

  if (!dir.exists(save_dir)) dir.create(save_dir, recursive = TRUE)

  ## build formula
  if (is.null(verbal_var)) {
    img_terms <- sprintf("%s * v2_sim_dis_test_z + %s * validity_binary_z + %s * tested_memorability_resmem_z",
                         visual_var, visual_var, visual_var)
    threeway  <- sprintf("%s * tested_memorability_resmem_z * v2_sim_dis_test_z",
                         visual_var)
  } else {
    img_terms <- sprintf("(%s + %s) * v2_sim_dis_test_z + (%s + %s) * validity_binary_z + (%s + %s) * tested_memorability_resmem_z",
                         visual_var, verbal_var, visual_var, verbal_var,
                         visual_var, verbal_var)
    threeway  <- sprintf("%s * tested_memorability_resmem_z * v2_sim_dis_test_z + %s * tested_memorability_resmem_z * v2_sim_dis_test_z",
                         visual_var, verbal_var)
  }

  common <- "tested_memorability_resmem_z * v2_sim_dis_test_z + 
             tested_memorability_resmem_z * validity_binary_z + 
             v2_sim_dis_test_z * validity_binary_z + 
             (1 | participant)"
  full   <- paste(img_terms, "+", threeway, "+", common)

  TAG        <- toupper(model_name_suffix)
  caption_of <- function(lab) paste0(TAG, ": ",
                     switch(lab, rt="RT Model", acc="Accuracy Model", errors="Error Type Model"))
  file_of    <- function(lab) file.path(save_dir,
                     sprintf("%s_%s_table.%s", model_name_suffix, lab, output))

  fit_print <- function(formula_str, data, family, label) {
    mod <- if (is.null(family)) lmer(as.formula(formula_str), data,
                                     control=lmerControl(optimizer="bobyqa"))
           else glmer(as.formula(formula_str), data, family,
                      control=glmerControl(optimizer="bobyqa"))
    assign(sprintf("%s_threeway_model_%s", model_name_suffix, label),
           mod, envir=.GlobalEnv)

    model_summary_table(mod,
      caption      = caption_of(label),
      p_cutoff     = p_cutoff,
      rename_terms = rename_terms,
      font_size    = font_size,
      caption_size = caption_size,
      save_as      = file_of(label),
      dpi          = dpi)
    invisible(mod)
  }

  list(
    rt       = fit_print(sprintf("log_rt ~ %s", full),        df_correct, NULL,     "rt"),
    accuracy = fit_print(sprintf("resp_correct ~ %s", full),  df,         binomial, "acc"),
    errors   = fit_print(sprintf("same_category_error ~ %s", full), df_errors, binomial, "errors")
  )
}


In [ ]:
# 2. OSIVQ Models (both visual and verbal)  
cat("\n\nRUNNING OSIVQ MODELS\n")
cat(rep("#", 60), "\n")
osivq_models <- run_imagery_models(
  visual_var = "z_osivq_visual_mean",
  verbal_var = "z_osivq_verbal_mean",
  model_name_suffix = "osivq"  
)

In [ ]:
# 3. VVIQ Models (visual only)
cat("\n\nRUNNING VVIQ MODELS\n")
cat(rep("#", 60), "\n")
vviq_models <- run_imagery_models(
  visual_var = "vviq_z",
  verbal_var = NULL,  # No verbal component for VVIQ
  model_name_suffix = "vviq"
)

In [ ]:
# The models are now available as:
# irq_model_rt, irq_model_acc, irq_model_errors
# osivq_model_rt, osivq_model_acc, osivq_model_errors  
# vviq_model_rt, vviq_model_acc, vviq_model_errors

cat("\n\nAll models completed and saved to global environment!\n")
cat("Model objects created:\n")
cat("- IRQ: irq_model_rt, irq_model_acc, irq_model_errors\n")
cat("- OSIVQ: osivq_model_rt, osivq_model_acc, osivq_model_errors\n") 
cat("- VVIQ: vviq_model_rt, vviq_model_acc, vviq_model_errors\n")

## <span style="color:#1a73e8">Regression Models</span>

<!-- <span style="color:black; font-size:18px"><em>.... <br> ... </em></span>
 -->

In [155]:
nice_names_irq <- c(
  z_irq_visual_mean             = "IRQ Visual",
  z_irq_verbal_mean             = "IRQ Verbal",
  v2_sim_dis_test_z             = "Visual Similarity (VGG-16)",
  tested_memorability_resmem_z  = "ResMem Scores",
  validity_binary_z             = "Validity"
)

# 1. IRQ Models with Three-Way Interactions
cat("RUNNING IRQ MODELS WITH THREE-WAY INTERACTIONS\n")
cat(rep("#", 70), "\n")
irq_threeway_models <- run_imagery_models_threeway(
  visual_var = "z_irq_visual_mean",
  verbal_var = "z_irq_verbal_mean", 
  model_name_suffix = "irq",
  rename_terms = nice_names_irq,
  save_dir = "tables",
  output = "png",
  dpi = 300
    
)

RUNNING IRQ MODELS WITH THREE-WAY INTERACTIONS
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 


Note that HTML color may not be displayed on PDF properly.



Predictor,b,SE,t,p,
(Intercept),0.3950066,0.0082182,48.0650196,0.0000000,***
IRQ Visual,0.0000972,0.0096108,0.0101133,0.9919309,
IRQ Verbal,0.0117632,0.0094700,1.2421568,0.2141840,
Visual Similarity (VGG-16),-0.0029665,0.0013142,-2.2571553,0.0240023,*
Validity,-0.0287008,0.0013103,-21.9033681,0.0000000,***
ResMem Scores,-0.0236142,0.0012707,-18.5834321,0.0000000,***
IRQ Visual × Visual Similarity (VGG-16),0.0037912,0.0015003,2.5270037,0.0115068,*
IRQ Verbal × Visual Similarity (VGG-16),-0.0037753,0.0014895,-2.5346028,0.0112602,*
IRQ Visual × Validity,-0.0034700,0.0015249,-2.2755973,0.0228739,*
IRQ Verbal × Validity,-0.0003427,0.0014839,-0.2309130,0.8173833,


Note that HTML color may not be displayed on PDF properly.



Predictor,OR,SE,z,p,
(Intercept),3.4461095,0.1536033,27.7577645,0.0000000,***
IRQ Visual,0.9009549,0.0470632,-1.9966695,0.0458611,*
IRQ Verbal,1.0187410,0.0524387,0.3607165,0.7183114,
Visual Similarity (VGG-16),1.0060772,0.0094626,0.6441790,0.5194594,
Validity,1.2310305,0.0106673,23.9864924,0.0000000,***
ResMem Scores,1.1657886,0.0103999,17.1953395,0.0000000,***
IRQ Visual × Visual Similarity (VGG-16),0.9908224,0.0112831,-0.8096547,0.4181386,
IRQ Verbal × Visual Similarity (VGG-16),1.0126571,0.0113515,1.1220372,0.2618466,
IRQ Visual × Validity,1.0185460,0.0106661,1.7548057,0.0792926,.
IRQ Verbal × Validity,0.9859107,0.0101232,-1.3819337,0.1669921,


Note that HTML color may not be displayed on PDF properly.



Predictor,OR,SE,z,p,
(Intercept),2.1258377,0.1019744,15.7219238,0.0000000,***
IRQ Visual,0.8926997,0.0471370,-2.1496045,0.0315865,*
IRQ Verbal,1.1293030,0.0591467,2.3217532,0.0202462,*
Visual Similarity (VGG-16),1.0298837,0.0191772,1.5813526,0.1137974,
Validity,1.2069136,0.0200136,11.3412960,0.0000000,***
ResMem Scores,0.9239065,0.0169914,-4.3034641,0.0000168,***
IRQ Visual × Visual Similarity (VGG-16),1.0156669,0.0220794,0.7151012,0.4745465,
IRQ Verbal × Visual Similarity (VGG-16),0.9704453,0.0205321,-1.4179571,0.1562033,
IRQ Visual × Validity,1.0274579,0.0193863,1.4356282,0.1511082,
IRQ Verbal × Validity,0.9764218,0.0184344,-1.2638349,0.2062893,


In [156]:
nice_names_osivq <- c(
    z_osivq_visual_mean           = "OSIVQ Visual",
    z_osivq_verbal_mean           = "OSIVQ Verbal",
    v2_sim_dis_test_z             = "Visual Similarity (VGG-16)",
    tested_memorability_resmem_z  = "ResMem Scores",
    validity_binary_z             = "Validity"
)

# 2. OSIVQ Models with Three-Way Interactions
cat("\n\nRUNNING OSIVQ MODELS WITH THREE-WAY INTERACTIONS\n")
cat(rep("#", 70), "\n")
osivq_threeway_models <- run_imagery_models_threeway(
  visual_var = "z_osivq_visual_mean",
  verbal_var = "z_osivq_verbal_mean",
  model_name_suffix = "osivq",
  rename_terms = nice_names_osivq,
  save_dir = "tables",
  output = "png",
  dpi = 300
)



RUNNING OSIVQ MODELS WITH THREE-WAY INTERACTIONS
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 


Note that HTML color may not be displayed on PDF properly.



Predictor,b,SE,t,p,
(Intercept),0.3952248,0.0081895,48.2599384,0.0000000,***
OSIVQ Visual,0.0111321,0.0084276,1.3209010,0.1865399,
OSIVQ Verbal,0.0064454,0.0084364,0.7639910,0.4448759,
Visual Similarity (VGG-16),-0.0032020,0.0013108,-2.4426753,0.0145820,*
Validity,-0.0289452,0.0013075,-22.1382877,0.0000000,***
ResMem Scores,-0.0236405,0.0012672,-18.6553628,0.0000000,***
OSIVQ Visual × Visual Similarity (VGG-16),0.0022001,0.0013090,1.6808088,0.0928057,.
OSIVQ Verbal × Visual Similarity (VGG-16),0.0015026,0.0013408,1.1206792,0.2624293,
OSIVQ Visual × Validity,-0.0030963,0.0013183,-2.3487062,0.0188423,*
OSIVQ Verbal × Validity,0.0037959,0.0013429,2.8266228,0.0047059,**


Note that HTML color may not be displayed on PDF properly.



Predictor,OR,SE,z,p,
(Intercept),3.4446927,0.1534458,27.7656105,0.0000000,***
OSIVQ Visual,0.9154145,0.0421142,-1.9210345,0.0547274,.
OSIVQ Verbal,0.9896582,0.0454243,-0.2264899,0.8208204,
Visual Similarity (VGG-16),1.0076933,0.0094570,0.8166232,0.4141438,
Validity,1.2324158,0.0106414,24.2021810,0.0000000,***
ResMem Scores,1.1655545,0.0103605,17.2346604,0.0000000,***
OSIVQ Visual × Visual Similarity (VGG-16),0.9923353,0.0100783,-0.7575923,0.4486951,
OSIVQ Verbal × Visual Similarity (VGG-16),0.9868215,0.0095342,-1.3730799,0.1697275,
OSIVQ Visual × Validity,1.0141258,0.0094559,1.5043652,0.1324873,
OSIVQ Verbal × Validity,0.9546062,0.0085140,-5.2087437,0.0000002,***


Note that HTML color may not be displayed on PDF properly.



Predictor,OR,SE,z,p,
(Intercept),2.1549697,0.1034764,15.9895036,0.0000000,***
OSIVQ Visual,0.9324652,0.0448630,-1.4533408,0.1461291,
OSIVQ Verbal,1.0239824,0.0485760,0.4995825,0.6173691,
Visual Similarity (VGG-16),1.0299325,0.0191484,1.5863519,0.1126595,
Validity,1.2040027,0.0198830,11.2420233,0.0000000,***
ResMem Scores,0.9247427,0.0169091,-4.2788625,0.0000188,***
OSIVQ Visual × Visual Similarity (VGG-16),0.9980927,0.0200756,-0.0949158,0.9243817,
OSIVQ Verbal × Visual Similarity (VGG-16),0.9941674,0.0180906,-0.3214696,0.7478545,
OSIVQ Visual × Validity,1.0383640,0.0184369,2.1202423,0.0339856,*
OSIVQ Verbal × Validity,0.9335167,0.0153407,-4.1864242,0.0000283,***


In [157]:
nice_names_vviq <- c(
    vviq_z                        = "VVIQ",
    v2_sim_dis_test_z             = "Visual Similarity (VGG-16)",
    tested_memorability_resmem_z  = "ResMem Scores",
    validity_binary_z             = "Validity"
)

# 3. VVIQ Models with Three-Way Interactions
cat("\n\nRUNNING VVIQ MODELS WITH THREE-WAY INTERACTIONS\n")
cat(rep("#", 70), "\n")
vviq_threeway_models <- run_imagery_models_threeway(
  visual_var = "vviq_z",
  verbal_var = NULL,  # No verbal component for VVIQ
  model_name_suffix = "vviq",
  rename_terms = nice_names,
  save_dir = "tables",
  output = "png",
  dpi = 300
)



RUNNING VVIQ MODELS WITH THREE-WAY INTERACTIONS
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 


Note that HTML color may not be displayed on PDF properly.



Predictor,b,SE,t,p,
(Intercept),0.3951030,0.0082462,47.9136242,0.0000000,***
vviq_z,0.0079359,0.0089171,0.8899620,0.3734902,
Visual Similarity (VGG-16),-0.0031514,0.0013159,-2.3948266,0.0166315,*
Validity,-0.0286152,0.0013114,-21.8204969,0.0000000,***
ResMem Memorability Scores,-0.0236466,0.0012725,-18.5833000,0.0000000,***
vviq_z × Visual Similarity (VGG-16),0.0008905,0.0013896,0.6408442,0.5216266,
vviq_z × Validity,-0.0029350,0.0013849,-2.1193569,0.0340648,*
vviq_z × ResMem Memorability Scores,0.0012519,0.0013565,0.9229401,0.3560425,
Visual Similarity (VGG-16) × ResMem Memorability Scores,-0.0044233,0.0012267,-3.6058997,0.0003113,***
Validity × ResMem Memorability Scores,-0.0024414,0.0013021,-1.8749829,0.0608003,.


Note that HTML color may not be displayed on PDF properly.



Predictor,OR,SE,z,p,
(Intercept),3.4566336,0.1550253,27.6551388,0.0000000,***
vviq_z,0.9223561,0.0448587,-1.6618491,0.0965430,.
Visual Similarity (VGG-16),1.0085236,0.0095191,0.8992290,0.3685307,
Validity,1.2308531,0.0106959,23.9023033,0.0000000,***
ResMem Memorability Scores,1.1654152,0.0104218,17.1178064,0.0000000,***
vviq_z × Visual Similarity (VGG-16),0.9850478,0.0104557,-1.4193010,0.1558113,
vviq_z × Validity,1.0060048,0.0098604,0.6108071,0.5413273,
vviq_z × ResMem Memorability Scores,0.9981935,0.0100282,-0.1799829,0.8571660,
Visual Similarity (VGG-16) × ResMem Memorability Scores,1.0447985,0.0089150,5.1359993,0.0000003,***
Validity × ResMem Memorability Scores,1.0123802,0.0085065,1.4643607,0.1430954,


Note that HTML color may not be displayed on PDF properly.



Predictor,OR,SE,z,p,
(Intercept),2.1508230,0.1046761,15.7362426,0.0000000,***
vviq_z,0.9685654,0.0489760,-0.6316413,0.5276213,
Visual Similarity (VGG-16),1.0273672,0.0191163,1.4510314,0.1467711,
Validity,1.2070201,0.0199890,11.3615653,0.0000000,***
ResMem Memorability Scores,0.9220747,0.0169070,-4.4246213,0.0000097,***
vviq_z × Visual Similarity (VGG-16),1.0181172,0.0204179,0.8953091,0.3706219,
vviq_z × Validity,1.0107072,0.0184063,0.5848198,0.5586689,
vviq_z × ResMem Memorability Scores,1.0157289,0.0199936,0.7928542,0.4278628,
Visual Similarity (VGG-16) × ResMem Memorability Scores,0.9447337,0.0165347,-3.2483263,0.0011609,**
Validity × ResMem Memorability Scores,1.0316486,0.0167119,1.9234349,0.0544255,.


In [106]:
# The three-way interaction models are now available as:
# irq_threeway_model_rt, irq_threeway_model_acc, irq_threeway_model_errors
# osivq_threeway_model_rt, osivq_threeway_model_acc, osivq_threeway_model_errors  
# vviq_threeway_model_rt, vviq_threeway_model_acc, vviq_threeway_model_errors

cat("\n\nAll three-way interaction models completed and saved to global environment!\n")
cat("Model objects created:\n")
cat("- IRQ: irq_threeway_model_rt, irq_threeway_model_acc, irq_threeway_model_errors\n")
cat("- OSIVQ: osivq_threeway_model_rt, osivq_threeway_model_acc, osivq_threeway_model_errors\n") 
cat("- VVIQ: vviq_threeway_model_rt, vviq_threeway_model_acc, vviq_threeway_model_errors\n")

cat("\nThree-way interactions included:\n")
cat("- IRQ: z_irq_visual_mean * tested_memorability_resmem_z * v2_sim_dis_test_z\n")
cat("       z_irq_verbal_mean * tested_memorability_resmem_z * v2_sim_dis_test_z\n")
cat("- OSIVQ: z_osivq_visual_mean * tested_memorability_resmem_z * v2_sim_dis_test_z\n")
cat("         z_osivq_verbal_mean * tested_memorability_resmem_z * v2_sim_dis_test_z\n")
cat("- VVIQ: vviq_z * tested_memorability_resmem_z * v2_sim_dis_test_z\n")



All three-way interaction models completed and saved to global environment!
Model objects created:
- IRQ: irq_threeway_model_rt, irq_threeway_model_acc, irq_threeway_model_errors
- OSIVQ: osivq_threeway_model_rt, osivq_threeway_model_acc, osivq_threeway_model_errors
- VVIQ: vviq_threeway_model_rt, vviq_threeway_model_acc, vviq_threeway_model_errors

Three-way interactions included:
- IRQ: z_irq_visual_mean * tested_memorability_resmem_z * v2_sim_dis_test_z
       z_irq_verbal_mean * tested_memorability_resmem_z * v2_sim_dis_test_z
- OSIVQ: z_osivq_visual_mean * tested_memorability_resmem_z * v2_sim_dis_test_z
         z_osivq_verbal_mean * tested_memorability_resmem_z * v2_sim_dis_test_z
- VVIQ: vviq_z * tested_memorability_resmem_z * v2_sim_dis_test_z
